In [ ]:
from riotwatcher import LolWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

lw = LolWatcher('RGAPI-ebb47b20-1588-4389-a7ea-29e2168af2ed')

my_region = 'americas'


me = lw.summoner.by_name('na1', 'satorian')

my_matches = lw.match.matchlist_by_puuid('americas', me['puuid'])

#print(my_matches)

n_games = 10
Games = {}
Game_duration=np.zeros(n_games)
Damage = np.zeros(n_games)
Gold = np.zeros(n_games)
Victory = np.zeros(n_games)
j=0
count=0
while count<n_games:
#     try:
        
        last_match = my_matches[count]
        match_detail = lw.match.by_id(my_region, last_match)
        #print(match_detail)
        participants = []
        for row in match_detail['info']['participants']:
            participants_row = {}
            participants_row['champion'] = row['championName']
            participants_row['win'] = row['win']
            participants_row['assists'] = row['assists']
            participants_row['gold'] = row['goldSpent']
            participants.append(participants_row)
        print(participants)
        Games[j] = pd.DataFrame(participants)
        i=0
        
        Games[j]= pd.DataFrame(participants)
#         for index, row in Games[j].iterrows():
#             print("HERE")
#             if row['Summoner_name']=='satorian':
#                 Damage[j]=row['totalDamageDealt']
#                 Gold[j]=row['goldEarned']
#                 Victory[j]=row['victory']
#                 print(Damage[j], Gold[j], Victory[j])
#         time.sleep(10)
#         j+=1
#         count+=1
#     except:
#         count+=1
        
dataset= pd.DataFrame()
dataset['Total Damage']= Damage
dataset['Gold']= Gold
dataset['Victory']=Victory #Boolean
#print(dataset)
train_dataset_raw = dataset.sample(frac=0.8,random_state=0)
test_dataset_raw = dataset.drop(train_dataset_raw.index)
train_dataset=train_dataset_raw.iloc[:,range(0,2)]
test_dataset=test_dataset_raw.iloc[:,range(0,2)]
train_labels=train_dataset_raw.iloc[:,2]
test_labels=test_dataset_raw.iloc[:,2]

sns.pairplot(train_dataset_raw, diag_kind="kde")